In [25]:
from bs4 import BeautifulSoup

In [2]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

In [3]:
soup = BeautifulSoup(html)

C:\Software\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Software\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


打印 `soup` 对象内容，格式输出

In [4]:
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>


## 四大对象

*BeautifulSoup* 将复杂的 `html` 文档转换成一个复杂的树形结构，每个节点都是 **Python** 对象，所有对象可分为四种。

- Tag
- NavigableString
- BeautifulSoup
- Comment

### **(1) Tag**

HTML 中的标签，例如：

``` html
<title>The Dormouse's story</title>
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>
```

In [5]:
print(soup.title)

<title>The Dormouse's story</title>


In [6]:
print(soup.head)

<head><title>The Dormouse's story</title></head>


In [7]:
print(soup.a)

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>


In [8]:
print(soup.p)

<p class="title" name="dromouse"><b>The Dormouse's story</b></p>


我们可以利用 `soup` 加标签名轻松获取这些标签的内容，但是，它查找的是所有内容中的**第一个**符合要求的标签。如何获取所有标签，前往下看。

In [9]:
print(type(soup.a))

<class 'bs4.element.Tag'>


对于 `Tag`，它有两个重要的属性，分别是 `name` 和 `attrs`，下面分别讲解。

#### name

In [10]:
print(soup.name)

[document]


In [11]:
print(soup.head.name)

head


`soup` 对象本身比较特殊，他的 `name` 即为 `[document]`，对于其他内部标签输出的值即为标签本身的名称。

#### attrs

In [12]:
soup.p.attrs

{'class': ['title'], 'name': 'dromouse'}

在这里，我们把 `<p>` 标签的所有属性打印输出了出来，得到类型字典。

如果需要某个属性，可以通过这种方式获取

In [13]:
soup.p['class']

['title']

`get()` 方法同样可以实现

In [14]:
soup.p.get('class')

['title']

你也可以对这些属性和内容进行修改，例如

In [15]:
soup.p['class'] = 'newClass'
soup.p

<p class="newClass" name="dromouse"><b>The Dormouse's story</b></p>

还可以对这个属性进行删除，例如

In [16]:
del soup.p['class']
soup.p

<p name="dromouse"><b>The Dormouse's story</b></p>

### (2) NavigableString

上面是关于标签本身的方法，如何获取标签中的内容，使用 `.string` 即可，例如

In [17]:
soup.p.string

"The Dormouse's story"

### (3) BeautifulSoup

*BeautifulSoup* 对象表示的是一个文档的全部内容，大多数时候，可以将他视为一个特殊的 `Tag` 对象，也可以获取对他的类型，名称，以及属性等。

In [18]:
type(soup.name)

<class 'str'>


In [19]:
soup.name

'[document]'

In [20]:
soup.attrs

{}

### (4) Comment

*Comment* 对象是一个特殊类型的NavigableString对象，其输出的内容仍然不包括注释符号，但是如果不好好处理，可能对我们的文本处理造成麻烦。

In [21]:
soup.a

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>

In [22]:
soup.a.string

' Elsie '

In [23]:
type(soup.a.string)

bs4.element.Comment

`<a>` 标签中的内容实际上是注释，但是如果我们利用 `.string` 来输出它的内容，可以看到注释符已经去除，无法分辨它和不同文本。

而且，当我们打印它的类型，发现它是一个 *Comment* 类型，所以，在使用前需要判断。

In [27]:
import bs4
if type(soup.a.string) == bs4.element.Comment:
    print(soup.a.string)

 Elsie 


## 遍历文档树

### （1）直接子节点

> 要点：`.contents` `.children` 属性

**`.contents`**

`tag` 的 `.content` 属性可以将 `tag` 的子节点以列表的方式输出

In [28]:
soup.head.contents

[<title>The Dormouse's story</title>]

可以通过切片索引访问它的每一个元素

In [29]:
soup.head.contents[0]

<title>The Dormouse's story</title>

**.children**

它返回的不是一个 list, 不过我们可以遍历获取所有的子节点。

也就是说它是一个 list 生成器对象

In [30]:
soup.head.children

In [31]:
for child in soup.body.children:
    print(child)



<p name="dromouse"><b>The Dormouse's story</b></p>


<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>


<p class="story">...</p>




### 所有子孙节点

> 知识点：`.descendants` 属性

**`.descendants`**

`.contents` 和 `.children` 属性仅包含 `tag` 的直接子节点，`.descendants` 属性可以对所有 `tag` 的子孙节点进行递归循环，和 `children` 类似，我们也需要遍历获取其中的内容。

In [32]:
for child in soup.descendants:
    print(child)

<html><head><title>The Dormouse's story</title></head>
<body>
<p name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body></html>
<head><title>The Dormouse's story</title></head>
<title>The Dormouse's story</title>
The Dormouse's story


<body>
<p name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
a

可以看出所有的节点都被打印出来了，包括 `<html>`

### (3) 节点内容

> 知识点： `.string` 属性

如果 `tag` 只有一个 *NavigableString* 类型子节点,那么这个 `tag` 可以使用 `.string` 得到子节点。如果一个 `tag` 仅有一个子节点,那么这个 `tag` 也可以使用 `.string` 方法, 输出结果与当前唯一子节点的 `.string` 结果相同。

通俗点说就是：如果一个标签里面没有标签了，那么 `.string` 就会返回标签里面的内容。如果标签里面只有唯一的一个标签了，那么 `.string` 也会返回最里面的内容。例如：

In [33]:
print(soup.head.string)

The Dormouse's story


In [34]:
soup.title.string

"The Dormouse's story"

如果 `tag` 包含了多个子节点，`tag` 就无法确定，`.string` 方法应该调用哪个子节点，返回值为 `None`

In [36]:
print(soup.html.string)

None


### (4) 多个内容

> 知识点：`.strings` `.stripped_strings` 属性

**`.strings`**

获取多个内容，不过需要遍历获取，比如下面的例子

In [37]:
for string in soup.strings:
    print(repr(string))

"The Dormouse's story"
'\n'
'\n'
"The Dormouse's story"
'\n'
'Once upon a time there were three little sisters; and their names were\n'
',\n'
'Lacie'
' and\n'
'Tillie'
';\nand they lived at the bottom of a well.'
'\n'
'...'
'\n'


**`.stripped_strings`**

输出的字符串中可能包含了许多空格或空行，使用 `.stripped_strings` 可以去除多余的内容

In [38]:
for string in soup.stripped_strings:
    print(repr(string))

"The Dormouse's story"
"The Dormouse's story"
'Once upon a time there were three little sisters; and their names were'
','
'Lacie'
'and'
'Tillie'
';\nand they lived at the bottom of a well.'
'...'


### (5) 父节点

> 知识点：`.parent` 属性

In [39]:
p = soup.p
p.parent.name

'body'

In [40]:
content = soup.head.title.string
content.parent.name

'title'

### （6）全部父节点

> 知识点：`.parents` 属性

通过元素的 `.parents` 属性可以递归得到元素的所有父辈节点，例如

In [42]:
content = soup.head.title.string
for parent in content.parents:
    print(parent.name)

title
head
html
[document]


### (7) 兄弟节点

> 知识点：`.next_sibling` `.previous_sibling` 属性

兄弟节点可以理解为和本节点处在同一级的节点，`.next_sibling` 属性获取了该节点的下一个兄弟点，`.previous_sibling` 则与之相反，如果节点不存在，则返回 `None`

**注意**：实际文档中的 `tag` 的 `.next_sibling` 和 `.previous_sibling` 属性通常是字符串或空白，因为空白或换行也可以被视作一个节点，所以得到的结果可能是空白或者换行

In [43]:
soup.p.next_sibling

'\n'

In [44]:
soup.p.next_sibling.next_sibling

<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

### (8) 全部兄弟节点

> 知识点：`.next_siblings` `.previous_siblings` 属性

通过 `.next_siblings` 和 `.previous_siblings` 属性可以对当前节点的兄弟节点迭代输出

In [45]:
for sibling in soup.a.next_siblings:
    print(repr(sibling))

',\n'
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
' and\n'
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
';\nand they lived at the bottom of a well.'


### （9） 前后节点

> 知识点：`.next_element` `.previous_element` 属性

与 `.next_sibling` `.previous_sibling` 不同，它并不是针对于兄弟节点，而是在所有节点，不分层次

比如 `<head>` 节点为

``` html
<head><title>The Dormouse's story</title></head>
```

那么它的下一个节点便是 `<title>`，它是不分层次关系的

In [46]:
soup.head.next_element

<title>The Dormouse's story</title>

### （10）所有前后节点

> 知识点：`.next_elements` `.previous_elements` 属性

通过 `.next_elements` 和 `.previous_elements` 的迭代器就可以向前或向后访问文档的解析内容，就好像文档正在被解析一样

In [47]:
for element in last_a_tag.next_elements:
    print(repr(element))

NameError: name 'last_a_tag' is not defined

## 搜索文档树

### （1）find_all(name, attrs, recursive, text, **kwargs)

`find_all()` 方法搜索当前 `tag` 的所有 `tag` 子节点，并判断是否符合过滤器的条件

#### 1) name 参数

name 参数可以查找所有名字为 name 的 tag，字符串对象会被自动忽略掉

A 传字符串

最简单的过滤器是字符串，在搜索方法中传入一个字符串参数，*BeautifulSoup* 会查找与字符串完整匹配的内容，下面的例子用于查找文档中所有的 `<b>` 标签

In [48]:
soup.find_all('b')

[<b>The Dormouse's story</b>]

In [49]:
soup.find_all('a')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

B 传正则表达式

如果传入正则表达式作为参数，*BeaurifulSoup* 会通过正则表达式的 `match()` 来匹配内容，下面例子中找出所有以 `b` 开头的标签，这表示 `<body>` 和 `<b>` 标签都应该被找到

In [50]:
import re
for tag in soup.find_all(re.compile('^b')):
    print(tag.name)

body
b


C 传列表

如果传入列表参数，*BeautifulSoup*会将与列表中任一元素匹配的内容返回。下面代码找到文档中所有 `<a>` 标签和 `<b>` 标签

In [51]:
soup.find_all(['a', 'b'])

[<b>The Dormouse's story</b>,
 <a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

D 传 True

True 可以匹配任何值,下面代码查找到所有的 `tag` ,但是不会返回字符串节点

In [52]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
title
body
p
b
p
a
a
a
p


E 传方法

如果没有合适过滤器，那么还可以定义一个方法，方法只接受一个元素参数，如果这个方法返回 `True` 表示当前元素匹配并且被找到，如果不是则反回 `False`

下面方法校验了当前元素，如果包含 `class` 属性却不包含 `id` 属性，那么将返回 `True`

In [53]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

In [54]:
soup.find_all(has_class_but_no_id)

[<p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>, <p class="story">...</p>]

#### 2) keyword 参数

> 注意：如果一个指定名字的参数不是搜索内置的参数名，搜索时会把该参数当作指定名字 `tag` 的属性来搜索，如果包含一个名字为 `id` 的参数，*BeautifulSoup* 会搜索每个 `tag` 的 `id` 属性

In [55]:
soup.find_all(id='link2')

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

如果传入 `href` 参数，*BeautifulSoup* 会搜索每个 `tag` 的 `href` 属性

In [56]:
soup.find_all(href=re.compile('elsie'))

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

使用多个指定名字的参数可以同时过滤 `tag` 的多个属性

In [57]:
soup.find_all(href=re.compile('elsie'), id='link1')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>]

在这里我们想用 `class` 过滤，不过 `class` 是 python 的关键词，这怎么办？加个下划线就可以

In [58]:
soup.find_all('a', class_='sister')

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

有些 `tag` 属性在搜索不能使用,比如 `HTML5` 中的 `data-*` 属性

In [66]:
data_soup = BeautifulSoup('div data-foo="value">foo!</div>')
# data_soup.find_all(data-foo='value')

C:\Software\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Software\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


但是可以通过 `find_all()` 方法的 `attrs` 参数定义一个字典参数来搜索包含特殊属性的  `tag`

In [67]:
data_soup.find_all(attrs={"data-foo": "value"})

[]

#### 3) text 参数

通过 `text` 参数可以搜索文档中的字符串内容。与 `name` 参数的可选值一样, `text` 参数接受 字符串 , 正则表达式 , 列表, True

In [68]:
soup.find_all(text='Elsie')

[]

In [69]:
soup.find_all(text=["Tillie", "Elsie", "Lacie"])

['Lacie', 'Tillie']

In [70]:
soup.find_all(text=re.compile("Dormouse"))

["The Dormouse's story", "The Dormouse's story"]

#### 4) limit 参数

`find_all()` 方法返回全部的搜索结构,如果文档树很大那么搜索会很慢.如果我们不需要全部结果,可以使用 `limit` 参数限制返回结果的数量.效果与 `SQL` 中的 `limit` 关键字类似,当搜索到的结果数量达到 `limit` 的限制时,就停止搜索返回结果.

文档树中有3个 `tag` 符合搜索条件,但结果只返回了2个,因为我们限制了返回数量

In [71]:
soup.find_all('a', limit=2)

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]